In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot as plt

import torch
import torch.nn as nn
from pytorch_tabnet.tab_model import TabNetClassifier
from tqdm import tqdm
from sklearn.metrics import f1_score ,roc_auc_score
import time
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

import lightgbm as lgb

In [2]:
path = '/Data2/folder_ti/data/'
train_csv = '1st_1011_train.csv'

train = pd.read_csv(path+train_csv,index_col=0)
train = train.astype({'gender':'object'})


In [19]:
train.head()

,application_id,user_id,purpose,is_applied,past.is_applied,m_past.is_applied,seq,min.loan_rate,product_n,n.limit_over_desire,...,employment_type,houseown_type,desired_amount,rehabilitation,existing_loan_cnt,logdata,diff,diff2,limit_over_desire_product,yd
83603,248576,9,사업자금,노대출,-1,0.465641,0.0,4.3,26,15,...,계약직,기타가족소유,15000000.0,2,1,3,0.0,0.000000,0.807692,0.517241
396875,1180616,11,생활비,대출,-1,0.465641,0.0,10.0,25,15,...,정규직,기타가족소유,2000000.0,-1,1,12,-1000000.0,-0.500000,0.960000,0.080000
588684,1751522,11,대환대출,대출,1,1.000000,1.0,10.7,25,15,...,정규직,기타가족소유,13000000.0,-1,1,21,31000000.0,2.384615,0.680000,0.520000
507188,1508745,14,대환대출,노대출,-1,0.465641,0.0,13.7,2,0,...,정규직,전월세,30000000.0,-1,0,8,-3000000.0,-0.100000,0.000000,1.111111
669142,1990215,17,생활비,대출,-1,0.465641,0.0,17.9,2,2,...,계약직,기타가족소유,3000000.0,-1,0,20,26000000.0,2.835714,1.000000,0.060000


In [3]:
path = '/Data2/folder_ti/data/'
second_csv = 'tr_2nd_classi_after_preprocessing.csv'

second = pd.read_csv(path+second_csv,index_col=0)

/opt/conda/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
second.is_applied.value_counts()

0    4208369
1     554711
Name: is_applied, dtype: int64

In [5]:
print(554711/(4208369+554711)*100)

11.646056753193312


In [6]:
100-11.64

88.36

In [4]:
second.drop(['purpose','income_type','rehabilitation','houseown_type','employment_type'],axis=1,inplace=True)
second = pd.merge(second, train.loc[:,['application_id','purpose','income_type','rehabilitation','houseown_type','employment_type']], on='application_id', how='left')
second.reset_index(drop=True, inplace=True)

In [5]:
train.head()

,application_id,user_id,purpose,is_applied,past.is_applied,m_past.is_applied,seq,min.loan_rate,product_n,n.limit_over_desire,...,employment_type,houseown_type,desired_amount,rehabilitation,existing_loan_cnt,logdata,diff,diff2,limit_over_desire_product,yd
83603,248576,9,사업자금,노대출,-1,0.465641,0.0,4.3,26,15,...,계약직,기타가족소유,15000000.0,2,1,3,0.0,0.000000,0.807692,0.517241
396875,1180616,11,생활비,대출,-1,0.465641,0.0,10.0,25,15,...,정규직,기타가족소유,2000000.0,-1,1,12,-1000000.0,-0.500000,0.960000,0.080000
588684,1751522,11,대환대출,대출,1,1.000000,1.0,10.7,25,15,...,정규직,기타가족소유,13000000.0,-1,1,21,31000000.0,2.384615,0.680000,0.520000
507188,1508745,14,대환대출,노대출,-1,0.465641,0.0,13.7,2,0,...,정규직,전월세,30000000.0,-1,0,8,-3000000.0,-0.100000,0.000000,1.111111
669142,1990215,17,생활비,대출,-1,0.465641,0.0,17.9,2,2,...,계약직,기타가족소유,3000000.0,-1,0,20,26000000.0,2.835714,1.000000,0.060000


In [6]:
second.head()

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,user_id,age,gender,...,rate_diff_with.min,relat.rate_diff_with.min,rate_diff_with.above,relat.rate_diff_with.above,logdata,purpose,income_type,rehabilitation,houseown_type,employment_type
0,14,2022-04-15 01:02:52,59,251,12000000.0,6.6,1,623310,25,1,...,0.0,0.000000,0.0,0.000000,2,대환대출,EARNEDINCOME,-1,기타가족소유,일용직
1,14,2022-04-15 01:03:08,49,136,12000000.0,6.6,1,623310,25,1,...,0.0,0.000000,0.0,0.000000,2,대환대출,EARNEDINCOME,-1,기타가족소유,일용직
2,14,2022-04-15 01:02:53,27,148,12000000.0,7.1,0,623310,25,1,...,0.5,0.075758,0.5,0.075758,2,대환대출,EARNEDINCOME,-1,기타가족소유,일용직
3,14,2022-04-15 01:02:53,33,189,12000000.0,7.9,0,623310,25,1,...,1.3,0.196970,0.8,0.112676,2,대환대출,EARNEDINCOME,-1,기타가족소유,일용직
4,14,2022-04-15 01:03:14,22,164,12000000.0,8.0,0,623310,25,1,...,1.4,0.212121,0.1,0.012658,2,대환대출,EARNEDINCOME,-1,기타가족소유,일용직


In [7]:
second.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4763080 entries, 0 to 4763079
Data columns (total 29 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   application_id              int64  
 1   loanapply_insert_time       object 
 2   bank_id                     int64  
 3   product_id                  int64  
 4   loan_limit                  float64
 5   loan_rate                   float64
 6   is_applied                  int64  
 7   user_id                     int64  
 8   age                         int64  
 9   gender                      int64  
 10  insert_time                 object 
 11  credit_score                int64  
 12  yearly_income               float64
 13  company_year                int64  
 14  desired_amount              float64
 15  existing_loan_cnt           float64
 16  n_product                   int64  
 17  rate_rank                   float64
 18  diff                        float64
 19  rate_diff_with.min   

In [11]:
second.nunique()

application_id            728862
loanapply_insert_time    2425396
bank_id                       62
product_id                   168
loan_limit                    70
loan_rate                    173
is_applied                     2
user_id                   259321
age                           73
gender                         3
insert_time               681142
credit_score                  81
yearly_income                 83
income_type                    6
company_year                  12
employment_type                4
houseown_type                  4
desired_amount               111
purpose                        8
rehabilitation                 4
existing_loan_cnt             10
rate_rank                    155
diff                        3272
rate_diff_with.min           195
rate_diff_with.above          41
logdata                       35
dtype: int64

In [14]:
for col in train.columns :
    print('col name & nan : ', col,'/',train[col].isnull().sum())

col name & nan :  application_id / 0
col name & nan :  user_id / 0
col name & nan :  purpose / 0
col name & nan :  is_applied / 0
col name & nan :  past.is_applied / 0
col name & nan :  m_past.is_applied / 0
col name & nan :  seq / 0
col name & nan :  min.loan_rate / 0
col name & nan :  product_n / 0
col name & nan :  n.limit_over_desire / 0
col name & nan :  loan_limit_min.rate / 0
col name & nan :  age / 0
col name & nan :  gender / 0
col name & nan :  credit_score / 0
col name & nan :  yearly_income / 0
col name & nan :  income_type / 0
col name & nan :  company_year / 0
col name & nan :  employment_type / 0
col name & nan :  houseown_type / 0
col name & nan :  desired_amount / 0
col name & nan :  rehabilitation / 0
col name & nan :  existing_loan_cnt / 0
col name & nan :  logdata / 0
col name & nan :  diff / 0
col name & nan :  diff2 / 0
col name & nan :  limit_over_desire_product / 0
col name & nan :  yd / 0


In [13]:
for col in second.columns :
    print('col name & nan : ', col,'/',second[col].isnull().sum())

col name & nan :  application_id / 0
col name & nan :  loanapply_insert_time / 0
col name & nan :  bank_id / 0
col name & nan :  product_id / 0
col name & nan :  loan_limit / 0
col name & nan :  loan_rate / 0
col name & nan :  is_applied / 0
col name & nan :  user_id / 0
col name & nan :  age / 0
col name & nan :  gender / 0
col name & nan :  insert_time / 0
col name & nan :  credit_score / 0
col name & nan :  yearly_income / 0
col name & nan :  company_year / 0
col name & nan :  desired_amount / 0
col name & nan :  rehabilitation / 0
col name & nan :  existing_loan_cnt / 0
col name & nan :  rate_rank / 0
col name & nan :  diff / 0
col name & nan :  rate_diff_with.min / 0
col name & nan :  rate_diff_with.above / 0
col name & nan :  logdata / 0
col name & nan :  houseown_type / 0
col name & nan :  income_type / 0
col name & nan :  employment_type / 0
col name & nan :  purpose / 0


In [16]:
second_ = second
second.drop(second_drop_col,axis=1,inplace=True)

In [8]:
train.head()

,application_id,user_id,purpose,is_applied,past.is_applied,m_past.is_applied,seq,min.loan_rate,product_n,n.limit_over_desire,...,employment_type,houseown_type,desired_amount,rehabilitation,existing_loan_cnt,logdata,diff,diff2,limit_over_desire_product,yd
83603,248576,9,사업자금,노대출,-1,0.465641,0.0,4.3,26,15,...,계약직,기타가족소유,15000000.0,2,1,3,0.0,0.000000,0.807692,0.517241
396875,1180616,11,생활비,대출,-1,0.465641,0.0,10.0,25,15,...,정규직,기타가족소유,2000000.0,-1,1,12,-1000000.0,-0.500000,0.960000,0.080000
588684,1751522,11,대환대출,대출,1,1.000000,1.0,10.7,25,15,...,정규직,기타가족소유,13000000.0,-1,1,21,31000000.0,2.384615,0.680000,0.520000
507188,1508745,14,대환대출,노대출,-1,0.465641,0.0,13.7,2,0,...,정규직,전월세,30000000.0,-1,0,8,-3000000.0,-0.100000,0.000000,1.111111
669142,1990215,17,생활비,대출,-1,0.465641,0.0,17.9,2,2,...,계약직,기타가족소유,3000000.0,-1,0,20,26000000.0,2.835714,1.000000,0.060000


In [9]:
def trans(x):
    if x == '노대출':
        return 0
    elif x == '대출':
        return 1
train['is_applied'] = train['is_applied'].apply(trans)

In [10]:
train.is_applied.value_counts()

1    377534
0    351328
Name: is_applied, dtype: int64

In [20]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 728862 entries, 83603 to 518912
Data columns (total 27 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   application_id             728862 non-null  int64  
 1   user_id                    728862 non-null  int64  
 2   purpose                    728862 non-null  object 
 3   is_applied                 728862 non-null  int64  
 4   past.is_applied            728862 non-null  int64  
 5   m_past.is_applied          728862 non-null  float64
 6   seq                        728862 non-null  float64
 7   min.loan_rate              728862 non-null  float64
 8   product_n                  728862 non-null  int64  
 9   n.limit_over_desire        728862 non-null  int64  
 10  loan_limit_min.rate        728862 non-null  float64
 11  age                        728862 non-null  int64  
 12  gender                     728862 non-null  object 
 13  credit_score             

In [58]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 728862 entries, 0 to 728861
Data columns (total 28 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   application_id             728862 non-null  int64  
 1   user_id                    728862 non-null  int64  
 2   purpose                    728862 non-null  int64  
 3   is_applied                 728862 non-null  int64  
 4   past.is_applied            728862 non-null  int64  
 5   m_past.is_applied          728862 non-null  float64
 6   seq                        728862 non-null  float64
 7   min.loan_rate              728862 non-null  float64
 8   product_n                  728862 non-null  int64  
 9   n.limit_over_desire        728862 non-null  int64  
 10  loan_limit_min.rate        728862 non-null  float64
 11  age                        728862 non-null  int64  
 12  gender                     728862 non-null  int64  
 13  credit_score               72

In [15]:
#train.drop(['set'],axis=1,inplace=True)
data_0 = train[train.is_applied == 0]
data_1 = train[train.is_applied == 1]

target = 'is_applied'    
if "set" not in data_0.columns:
    data_0["set"] = np.random.choice(["train", "valid"], p =[.8, .2], size=(data_0.shape[0],))
if "set" not in data_1.columns:
    data_1["set"] = np.random.choice(["train", "valid"], p =[.8, .2], size=(data_1.shape[0],))

/tmp/ipykernel_5702/4157980594.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_0["set"] = np.random.choice(["train", "valid"], p =[.8, .2], size=(data_0.shape[0],))
/tmp/ipykernel_5702/4157980594.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_1["set"] = np.random.choice(["train", "valid"], p =[.8, .2], size=(data_1.shape[0],))


In [17]:
train = pd.concat([data_0,data_1], ignore_index=True)
train = train.sample(frac=1).reset_index(drop=True)

In [19]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 728862 entries, 0 to 728861
Data columns (total 28 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   application_id             728862 non-null  int64  
 1   user_id                    728862 non-null  int64  
 2   purpose                    728862 non-null  object 
 3   is_applied                 728862 non-null  int64  
 4   past.is_applied            728862 non-null  int64  
 5   m_past.is_applied          728862 non-null  float64
 6   seq                        728862 non-null  float64
 7   min.loan_rate              728862 non-null  float64
 8   product_n                  728862 non-null  int64  
 9   n.limit_over_desire        728862 non-null  int64  
 10  loan_limit_min.rate        728862 non-null  float64
 11  age                        728862 non-null  int64  
 12  gender                     728862 non-null  object 
 13  credit_score               72

In [27]:
cat_col = ['purpose','income_type','employment_type','houseown_type','rehabilitation']

In [30]:
cat_trans = dict()
for cat in cat_col :
    cat_trans[cat] = dict()
    type_ = list(set(train[cat].tolist()))
    for i, ty in enumerate(type_) :
        cat_trans[cat][ty] = i

NameError: name 'train_idxs' is not defined

In [29]:
cat_trans

{'bank_id': {}}

In [23]:
from tqdm import tqdm
for cat in cat_col :
#    if cat == 'gender' :
#        continue
    for k in tqdm(cat_trans[cat].keys()) :
        train[cat][train[cat]==k] = cat_trans[cat][k]   

  0%|                                                                                                                       | 0/8 [00:00<?, ?it/s]/tmp/ipykernel_5702/1901354475.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[cat][train[cat]==k] = cat_trans[cat][k]
  0%|                                                                                                                       | 0/4 [00:00<?, ?it/s]/tmp/ipykernel_5702/1901354475.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[cat][train[cat]==k] = cat_trans[cat][k]
/tmp/ipykernel_5702/1901354475.py:6: SettingWithCopyWarning: 
A value is try

In [24]:
train_indices = train[train.set=="train"].index
valid_indices = train[train.set=="valid"].index

In [26]:
train.columns

Index(['application_id', 'user_id', 'purpose', 'is_applied', 'past.is_applied',
       'm_past.is_applied', 'seq', 'min.loan_rate', 'product_n',
       'n.limit_over_desire', 'loan_limit_min.rate', 'age', 'gender',
       'credit_score', 'yearly_income', 'income_type', 'company_year',
       'employment_type', 'houseown_type', 'desired_amount', 'rehabilitation',
       'existing_loan_cnt', 'logdata', 'diff', 'diff2',
       'limit_over_desire_product', 'yd', 'set'],
      dtype='object')

In [78]:
types = train.dtypes
target = 'is_applied'
unused_col = ['set','user_id','application_id']
cat_col = ['purpose','income_type','rehabilitation','gender','employment_type','houseown_type']

features = [ col for col in train.columns if col not in unused_col+[target]] 

In [27]:
#nunique = train.nunique()
types = train.dtypes
target = 'is_applied'
unused_col = ['set','user_id','application_id']
cat_col = ['purpose','income_type','rehabilitation','gender','employment_type','houseown_type']
categorical_dims =  {}
for col in train.columns:
    if col in cat_col :
        print(col, train[col].nunique())
        l_enc = LabelEncoder()
        train[col] = l_enc.fit_transform(train[col].values)
        categorical_dims[col] = len(l_enc.classes_)
    elif col not in unused_col+[target]:
        train.fillna(train.loc[:, col].mean(), inplace=True)


# Categorical Embedding을 위해 Categorical 변수의 차원과 idxs를 담음.
features = [ col for col in train.columns if col not in unused_col+[target]] 
cat_idxs = [ i for i, f in enumerate(features) if f in cat_col]
cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in cat_col]
#train_shuf = train_shuf.astype({'income_type':'int64','employment_type':'int64','houseown_type':'int64','purpose':'int64'})

purpose 8
gender 3
income_type 6
employment_type 4
houseown_type 4
rehabilitation 4


In [28]:
id_set = list(set(train.application_id.to_list()))

In [29]:
valid_id_set = list(set(train[train.set == 'valid'].application_id.to_list()))
train_id_set = list(set(train[train.set == 'train'].application_id.to_list()))

In [30]:
len(valid_id_set)

145801

In [31]:
train.head()

,application_id,user_id,purpose,is_applied,past.is_applied,m_past.is_applied,seq,min.loan_rate,product_n,n.limit_over_desire,...,houseown_type,desired_amount,rehabilitation,existing_loan_cnt,logdata,diff,diff2,limit_over_desire_product,yd,set
0,422303,867441,6,1,-1,0.465641,0.0,8.5,1,1,...,0,5000000.0,3,0,4,22000000.0,2.835714,1.000000,0.208333,train
1,1766400,111137,1,0,0,0.000000,2.0,15.4,5,1,...,0,20000000.0,3,6,13,-17000000.0,-0.850000,0.200000,1.333333,train
2,1407706,92500,1,1,1,1.000000,3.0,13.3,11,9,...,2,10000000.0,3,10,11,13000000.0,1.300000,0.818182,0.370370,valid
3,25415,222392,5,0,-1,0.465641,0.0,9.9,4,0,...,1,30000000.0,3,0,10,-27000000.0,-0.900000,0.000000,0.857143,train
4,1378329,103673,5,0,-1,0.465641,0.0,6.5,40,15,...,1,10000000.0,3,1,5,-1000000.0,-0.100000,0.600000,0.303030,train


In [32]:
features

['purpose',
 'past.is_applied',
 'm_past.is_applied',
 'seq',
 'min.loan_rate',
 'product_n',
 'n.limit_over_desire',
 'loan_limit_min.rate',
 'age',
 'gender',
 'credit_score',
 'yearly_income',
 'income_type',
 'company_year',
 'employment_type',
 'houseown_type',
 'desired_amount',
 'rehabilitation',
 'existing_loan_cnt',
 'logdata',
 'diff',
 'diff2',
 'limit_over_desire_product',
 'yd']

In [33]:

X_train = train[features].values[train_indices]
y_train = train[target].values[train_indices]

X_valid = train[features].values[valid_indices]
y_valid = train[target].values[valid_indices]

In [34]:
import torch
DEVICE = 'cuda:3' if torch.cuda.is_available() else 'cpu'
print("Using {}".format(DEVICE))

Using cuda:3


In [35]:
X_train.shape

(583061, 24)

In [36]:

params = {'n_d' : 4,
         'n_steps' :5,        # tree에 depth
         'gamma':1.5,
         'n_independent':2,
          'cat_idxs':cat_idxs,
          'cat_dims':cat_dims,
         'n_shared':2,
         'cat_emb_dim':1,
         'lambda_sparse':1e-4,
         'momentum':0.5,
         'clip_value':2.,
         'optimizer_fn':torch.optim.Adam,
         'optimizer_params':dict(lr=2e-2),
         'scheduler_params':{"gamma": 0.95,
                         "step_size": 20},
         'scheduler_fn':torch.optim.lr_scheduler.StepLR,
         'epsilon':1e-15,
         'device_name':DEVICE,
         'mask_type':'sparsemax'}

clf = TabNetClassifier(**params
)
s = time.time()

clf.fit(X_train=X_train, y_train=y_train,eval_set=[(X_valid, y_valid)],
    max_epochs=50 , patience=20,
    batch_size=10240, virtual_batch_size=1280,
    num_workers=0,
    weights=1,
    drop_last=False
)

print('TabNet gpu sec : ',time.time()- s)


pred_valid = clf.predict(X_valid)


f1_valid = f1_score(y_valid, pred_valid) # y_true : 정답 값, y_pred : 예측 값 
print("Valid F1 score: {}".format(f1_valid))
 

from sklearn.metrics import roc_auc_score 
roc_auc = roc_auc_score(y_valid, pred_valid)
print("Valid ROC AUC Score: {}".format(roc_auc))

/opt/conda/lib/python3.8/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cuda:3
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.68376 | val_0_auc: 0.59642 |  0:00:12s
epoch 1  | loss: 0.64077 | val_0_auc: 0.66471 |  0:00:25s
epoch 2  | loss: 0.6352  | val_0_auc: 0.68104 |  0:00:38s
epoch 3  | loss: 0.63342 | val_0_auc: 0.69538 |  0:00:50s
epoch 4  | loss: 0.62998 | val_0_auc: 0.70032 |  0:01:03s
epoch 5  | loss: 0.62764 | val_0_auc: 0.70632 |  0:01:15s
epoch 6  | loss: 0.62714 | val_0_auc: 0.70678 |  0:01:28s
epoch 7  | loss: 0.62351 | val_0_auc: 0.70987 |  0:01:40s
epoch 8  | loss: 0.62556 | val_0_auc: 0.70312 |  0:01:52s
epoch 9  | loss: 0.62698 | val_0_auc: 0.70613 |  0:02:04s
epoch 10 | loss: 0.62584 | val_0_auc: 0.70737 |  0:02:17s
epoch 11 | loss: 0.62574 | val_0_auc: 0.70235 |  0:02:30s
epoch 12 | loss: 0.62744 | val_0_auc: 0.70389 |  0:02:42s
epoch 13 | loss: 0.62681 | val_0_auc: 0.70728 |  0:02:55s
epoch 14 | loss: 0.62631 | val_0_auc: 0.70517 |  0:03:07s
epoch 15 | loss: 0.6277  | val_0_auc: 0.70596 |  0:03:19s
epoch 16 | loss: 0.62568 | val_0_auc: 0.70823 |  0:03:32s
epoch 17 | los

/opt/conda/lib/python3.8/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


TabNet gpu sec :  1009.1102468967438
Valid F1 score: 0.6866554498133446
Valid ROC AUC Score: 0.6723381246744304


In [37]:
pred_prob_valid = clf.predict_proba(X_valid)

def thres_optimization(threshold, array) :
    pred = np.ceil(array[:,1]-threshold)
    return pred

score_valid = [0]
best_thres_valid = 0
for i in range(1,100) :
    pre_valid = thres_optimization(i/100,pred_prob_valid)
    
    f1_valid = f1_score(y_valid, pre_valid)
    
    score_valid.append(f1_valid)
    
best_thres_valid = score_valid.index(max(score_valid))/100
print(max(score_valid))
print(best_thres_valid)

    

0.7238566340963735
0.33


# 2nd Classification

In [73]:
valid_id_set = list(set(train[train.set == 'valid'].application_id.to_list()))
train_id_set = list(set(train[(train.set == 'train')&(train.is_applied == 1)].application_id.to_list()))

In [74]:
print(len(valid_id_set)+len(train_id_set))

447530


In [112]:
valid_second = second[second['application_id'].isin(valid_id_set)]
train_second = second[second['application_id'].isin(train_id_set)]

In [41]:
pred_valid = thres_optimization(0.33,pred_prob_valid)

In [42]:
valid_set = train.loc[valid_indices,['application_id']]
valid_set['pred'] = pred_valid

In [43]:
valid_set.head()

,application_id,pred
2,1407706,1.0
16,2008340,-0.0
39,258014,1.0
43,636053,1.0
49,2139226,1.0


In [44]:
second.head()

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,user_id,age,gender,...,rate_diff_with.min,relat.rate_diff_with.min,rate_diff_with.above,relat.rate_diff_with.above,logdata,purpose,income_type,rehabilitation,houseown_type,employment_type
0,14,2022-04-15 01:02:52,59,251,12000000.0,6.6,1,623310,25,1,...,0.0,0.000000,0.0,0.000000,2,대환대출,EARNEDINCOME,-1,기타가족소유,일용직
1,14,2022-04-15 01:03:08,49,136,12000000.0,6.6,1,623310,25,1,...,0.0,0.000000,0.0,0.000000,2,대환대출,EARNEDINCOME,-1,기타가족소유,일용직
2,14,2022-04-15 01:02:53,27,148,12000000.0,7.1,0,623310,25,1,...,0.5,0.075758,0.5,0.075758,2,대환대출,EARNEDINCOME,-1,기타가족소유,일용직
3,14,2022-04-15 01:02:53,33,189,12000000.0,7.9,0,623310,25,1,...,1.3,0.196970,0.8,0.112676,2,대환대출,EARNEDINCOME,-1,기타가족소유,일용직
4,14,2022-04-15 01:03:14,22,164,12000000.0,8.0,0,623310,25,1,...,1.4,0.212121,0.1,0.012658,2,대환대출,EARNEDINCOME,-1,기타가족소유,일용직


In [75]:
valid_2nd = pd.merge(valid_second, valid_set, on='application_id', how='left')
train_2nd = train_second
train_2nd.head()

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied,user_id,age,gender,...,rate_diff_with.min,relat.rate_diff_with.min,rate_diff_with.above,relat.rate_diff_with.above,logdata,purpose,income_type,rehabilitation,houseown_type,employment_type
0,14,2022-04-15 01:02:52,56,251,12000000.0,6.6,1,623310,25,1,...,0.0,0.000000,0.0,0.000000,2,2,4,-1,2,0
1,14,2022-04-15 01:03:08,47,136,12000000.0,6.6,1,623310,25,1,...,0.0,0.000000,0.0,0.000000,2,2,4,-1,2,0
2,14,2022-04-15 01:02:53,26,148,12000000.0,7.1,0,623310,25,1,...,0.5,0.075758,0.5,0.075758,2,2,4,-1,2,0
3,14,2022-04-15 01:02:53,32,189,12000000.0,7.9,0,623310,25,1,...,1.3,0.196970,0.8,0.112676,2,2,4,-1,2,0
4,14,2022-04-15 01:03:14,21,164,12000000.0,8.0,0,623310,25,1,...,1.4,0.212121,0.1,0.012658,2,2,4,-1,2,0


In [116]:
train_2nd = train_second

In [76]:
test_2nd = valid_2nd[valid_2nd.pred == 1]

In [142]:
train_2nd.to_csv('/Data2/folder_ti/data/2nd/train_2nd.csv')
valid_2nd.to_csv('/Data2/folder_ti/data/2nd/valid_2nd.csv')
test_2nd.to_csv('/Data2/folder_ti/data/2nd/test_2nd.csv')

### 한번 끊고 시작

In [8]:
path2 = '/Data2/folder_ti/data/'
second_csv = 'tr_2nd_class_after_bank_cluster5.csv'


second = pd.read_csv(path2+second_csv)




In [12]:
second.columns

Index(['application_id', 'loanapply_insert_time', 'bank_id', 'product_id',
       'loan_limit', 'loan_rate', 'is_applied', 'user_id', 'age', 'gender',
       'insert_time', 'credit_score', 'yearly_income', 'income_type',
       'company_year', 'employment_type', 'houseown_type', 'desired_amount',
       'purpose', 'rehabilitation', 'existing_loan_cnt', 'n_product',
       'rate_rank', 'diff', 'rate_diff_with.min', 'relat.rate_diff_with.min',
       'same_rank_length', 'rate_diff_with.above',
       'relat.rate_diff_with.above', 'logdata', 'bank_cluster',
       'cluster_min_rate'],
      dtype='object')

In [14]:
a = [1,2,3]
b = [4,4,5]
c = [3,4]
a+b+c

[1, 2, 3, 4, 4, 5, 3, 4]

In [15]:
a = list(set(second[second['is_applied']==1].application_id.tolist()))
print(len(a))

377506


In [2]:
path = '/Data2/folder_ti/data/2nd/'
path2 = '/Data2/folder_ti/data/'
train_2nd_csv = 'train_2nd.csv'
valid_2nd_csv = 'valid_2nd.csv'
test_2nd_csv = 'test_2nd.csv'
second_csv = 'tr_2nd_class_after_bank_cluster5.csv'
first_csv = '1st_1011_train.csv'

first = pd.read_csv(path2+first_csv,index_col=0)
second = pd.read_csv(path2+second_csv)
second.reset_index(drop=True,inplace=True)
train_2nd = pd.read_csv(path+train_2nd_csv,index_col=0)
valid_2nd = pd.read_csv(path+valid_2nd_csv,index_col=0)
test_2nd = pd.read_csv(path+test_2nd_csv,index_col=0)

/opt/conda/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
print(first.columns)
print(second.columns)
print(train_2nd.columns)
print(valid_2nd.columns)
print(test_2nd.columns)

Index(['application_id', 'user_id', 'purpose', 'is_applied', 'past.is_applied',
       'm_past.is_applied', 'seq', 'min.loan_rate', 'product_n',
       'n.limit_over_desire', 'loan_limit_min.rate', 'age', 'gender',
       'credit_score', 'yearly_income', 'income_type', 'company_year',
       'employment_type', 'houseown_type', 'desired_amount', 'rehabilitation',
       'existing_loan_cnt', 'logdata', 'diff', 'diff2',
       'limit_over_desire_product', 'yd'],
      dtype='object')
Index(['application_id', 'loanapply_insert_time', 'bank_id', 'product_id',
       'loan_limit', 'loan_rate', 'is_applied', 'user_id', 'age', 'gender',
       'insert_time', 'credit_score', 'yearly_income', 'income_type',
       'company_year', 'employment_type', 'houseown_type', 'desired_amount',
       'purpose', 'rehabilitation', 'existing_loan_cnt', 'n_product',
       'rate_rank', 'diff', 'rate_diff_with.min', 'relat.rate_diff_with.min',
       'same_rank_length', 'rate_diff_with.above',
       'relat.rat

In [4]:
print(first.info())
print(second.info())
print(train_2nd.info())
print(valid_2nd.info())
print(test_2nd.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 728862 entries, 83603 to 518912
Data columns (total 27 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   application_id             728862 non-null  int64  
 1   user_id                    728862 non-null  int64  
 2   purpose                    728862 non-null  object 
 3   is_applied                 728862 non-null  object 
 4   past.is_applied            728862 non-null  int64  
 5   m_past.is_applied          728862 non-null  float64
 6   seq                        728862 non-null  float64
 7   min.loan_rate              728862 non-null  float64
 8   product_n                  728862 non-null  int64  
 9   n.limit_over_desire        728862 non-null  int64  
 10  loan_limit_min.rate        728862 non-null  float64
 11  age                        728862 non-null  int64  
 12  gender                     728862 non-null  int64  
 13  credit_score             

In [29]:
print(second.shape[0])
print(valid_2nd.shape[0]+train_2nd.shape[0])
print(valid_2nd.is_applied.value_counts())
print(test_2nd.is_applied.value_counts())
print( (set(second.application_id.tolist()) - set(train_2nd.application_id.tolist())) - set(valid_2nd.application_id.tolist()) )

4763080
4763080
0    843984
1    111274
Name: is_applied, dtype: int64
0    716854
1    101737
Name: is_applied, dtype: int64
set()


In [11]:
print(first.purpose.iloc[0])
print(second.purpose.iloc[0])
print(train_2nd.purpose.iloc[0])
print(valid_2nd.purpose.iloc[0])
print(test_2nd.purpose.iloc[0])

사업자금
��ȯ����
2
3
3


In [11]:
fa = second[second['application_id'].isin(list(set(valid_2nd.application_id.tolist())))]

In [27]:
valid_2nd.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)

In [14]:
fa.shape[0]

955258

In [6]:
second.drop(['purpose','employment_type','houseown_type'],axis=1,inplace=True)

In [7]:
second = pd.merge(second,first.loc[:,['application_id','purpose','employment_type','houseown_type']],on='application_id',how='left')

In [8]:
valid_pred = valid_2nd.loc[:,['application_id','pred']]

train_2nd = second[second['application_id'].isin(list(set(train_2nd.application_id.tolist())))]
valid_2nd = pd.merge(second[second['application_id'].isin(list(set(valid_2nd.application_id.tolist())))],valid_pred,on='application_id',how='left')
test_2nd = second[second['application_id'].isin(list(set(test_2nd.application_id.tolist())))]

In [65]:
print(train_2nd.shape[0])
print(valid_2nd.shape[0])
print(test_2nd.shape[0])

3807822
955258
818591


In [15]:
second.columns

Index(['application_id', 'loanapply_insert_time', 'bank_id', 'product_id',
       'loan_limit', 'loan_rate', 'is_applied', 'user_id', 'age', 'gender',
       'insert_time', 'credit_score', 'yearly_income', 'income_type',
       'company_year', 'desired_amount', 'rehabilitation', 'existing_loan_cnt',
       'n_product', 'rate_rank', 'diff', 'rate_diff_with.min',
       'relat.rate_diff_with.min', 'same_rank_length', 'rate_diff_with.above',
       'relat.rate_diff_with.above', 'logdata', 'bank_cluster',
       'cluster_min_rate', 'purpose', 'employment_type', 'houseown_type'],
      dtype='object')

In [16]:
first.columns

Index(['application_id', 'user_id', 'purpose', 'is_applied', 'past.is_applied',
       'm_past.is_applied', 'seq', 'min.loan_rate', 'product_n',
       'n.limit_over_desire', 'loan_limit_min.rate', 'age', 'gender',
       'credit_score', 'yearly_income', 'income_type', 'company_year',
       'employment_type', 'houseown_type', 'desired_amount', 'rehabilitation',
       'existing_loan_cnt', 'logdata', 'diff', 'diff2',
       'limit_over_desire_product', 'yd'],
      dtype='object')

In [30]:
first_only_col = ['application_id','past.is_applied','m_past.is_applied','seq','min.loan_rate','product_n','n.limit_over_desire','loan_limit_min.rate',
                 'diff2','limit_over_desire_product','yd']

In [31]:
train_2nd = pd.merge(train_2nd,first[first['application_id'].isin(list(set(train_2nd.application_id.tolist())))].loc[:,first_only_col],on='application_id', how='left')
valid_2nd = pd.merge(valid_2nd,first[first['application_id'].isin(list(set(valid_2nd.application_id.tolist())))].loc[:,first_only_col],on='application_id', how='left')
test_2nd = pd.merge(test_2nd,first[first['application_id'].isin(list(set(test_2nd.application_id.tolist())))].loc[:,first_only_col],on='application_id', how='left')

In [32]:
print(train_2nd.shape[0])
print(valid_2nd.shape[0])
print(test_2nd.shape[0])

3807822
955258
818591


In [16]:
second.columns

Index(['application_id', 'loanapply_insert_time', 'bank_id', 'product_id',
       'loan_limit', 'loan_rate', 'is_applied', 'user_id', 'age', 'gender',
       'insert_time', 'credit_score', 'yearly_income', 'income_type',
       'company_year', 'employment_type', 'houseown_type', 'desired_amount',
       'purpose', 'rehabilitation', 'existing_loan_cnt', 'n_product',
       'rate_rank', 'diff', 'rate_diff_with.min', 'relat.rate_diff_with.min',
       'same_rank_length', 'rate_diff_with.above',
       'relat.rate_diff_with.above', 'logdata', 'bank_cluster',
       'cluster_min_rate'],
      dtype='object')

In [33]:
unused_col = ['user_id','loanapply_insert_time','insert_time','gender',
             'company_year','product_id']
train_2nd.drop(unused_col,axis=1,inplace=True)
valid_2nd.drop(unused_col,axis=1,inplace=True)
test_2nd.drop(unused_col,axis=1,inplace=True)

In [22]:
test_2nd.columns

Index(['application_id', 'bank_id', 'loan_limit', 'loan_rate', 'is_applied',
       'age', 'credit_score', 'income_type', 'employment_type',
       'houseown_type', 'desired_amount', 'purpose', 'rehabilitation',
       'n_product', 'rate_rank', 'diff', 'rate_diff_with.min',
       'relat.rate_diff_with.min', 'same_rank_length', 'rate_diff_with.above',
       'relat.rate_diff_with.above', 'logdata', 'bank_cluster',
       'cluster_min_rate', 'past.is_applied', 'm_past.is_applied', 'seq',
       'min.loan_rate', 'product_n', 'n.limit_over_desire',
       'loan_limit_min.rate', 'diff2', 'limit_over_desire_product', 'yd'],
      dtype='object')

In [34]:
cat_col = ['bank_id','purpose','income_type','employment_type','houseown_type','rehabilitation','rate_rank','bank_cluster','cluster_min_rate']

In [35]:
cat_trans = dict()
for cat in cat_col :
    cat_trans[cat] = dict()
    type_ = list(set(train_2nd[cat].tolist()))
    for i, ty in enumerate(type_) :
        cat_trans[cat][ty] = i
cat_trans1 = dict()
for cat in cat_col :
    cat_trans1[cat] = dict()
    type_ = list(set(valid_2nd[cat].tolist()))
    for i, ty in enumerate(type_) :
        cat_trans1[cat][ty] = i
        
cat_trans2 = dict()
for cat in cat_col :
    cat_trans2[cat] = dict()
    type_ = list(set(test_2nd[cat].tolist()))
    for i, ty in enumerate(type_) :
        cat_trans2[cat][ty] = i

In [70]:
cat_trans2

{'bank_id': {1: 0,
  2: 1,
  3: 2,
  4: 3,
  5: 4,
  6: 5,
  7: 6,
  8: 7,
  9: 8,
  10: 9,
  11: 10,
  12: 11,
  13: 12,
  14: 13,
  15: 14,
  16: 15,
  17: 16,
  18: 17,
  19: 18,
  20: 19,
  21: 20,
  22: 21,
  23: 22,
  24: 23,
  25: 24,
  26: 25,
  27: 26,
  28: 27,
  29: 28,
  30: 29,
  31: 30,
  32: 31,
  33: 32,
  34: 33,
  35: 34,
  36: 35,
  37: 36,
  38: 37,
  39: 38,
  40: 39,
  41: 40,
  42: 41,
  43: 42,
  44: 43,
  45: 44,
  46: 45,
  47: 46,
  49: 47,
  50: 48,
  51: 49,
  52: 50,
  54: 51,
  55: 52,
  56: 53,
  57: 54,
  58: 55,
  59: 56,
  60: 57,
  61: 58,
  62: 59,
  63: 60,
  64: 61},
 'purpose': {'자동차구입': 0,
  '사업자금': 1,
  '대환대출': 2,
  '생활비': 3,
  '투자': 4,
  '기타': 5,
  '주택구입': 6,
  '전월세보증금': 7},
 'income_type': {'EARNEDINCOME': 0,
  'PRIVATEBUSINESS': 1,
  'FREELANCER': 2,
  'EARNEDINCOME2': 3,
  'OTHERINCOME': 4,
  'PRACTITIONER': 5},
 'employment_type': {'계약직': 0, '기타': 1, '일용직': 2, '정규직': 3},
 'houseown_type': {'배우자': 0, '전월세': 1, '기타가족소유': 2, '자가': 3},
 'rehab

In [73]:
train_2nd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3807822 entries, 0 to 3807821
Data columns (total 34 columns):
 #   Column                      Dtype  
---  ------                      -----  
 0   application_id              int64  
 1   bank_id                     int64  
 2   loan_limit                  float64
 3   loan_rate                   float64
 4   is_applied                  int64  
 5   age                         int64  
 6   credit_score                int64  
 7   income_type                 object 
 8   desired_amount              float64
 9   rehabilitation              int64  
 10  n_product                   int64  
 11  rate_rank                   int64  
 12  diff                        float64
 13  rate_diff_with.min          float64
 14  relat.rate_diff_with.min    float64
 15  same_rank_length            int64  
 16  rate_diff_with.above        float64
 17  relat.rate_diff_with.above  float64
 18  logdata                     int64  
 19  bank_cluster         

In [36]:
from tqdm import tqdm
for cat in cat_col :
    for k in tqdm(cat_trans[cat].keys()) :
        train_2nd[cat][train_2nd[cat]==k] = cat_trans[cat][k]   
for cat in cat_col :
    for k in tqdm(cat_trans1[cat].keys()) :
        valid_2nd[cat][valid_2nd[cat]==k] = cat_trans1[cat][k]   
for cat in cat_col :
    for k in tqdm(cat_trans2[cat].keys()) :
        test_2nd[cat][test_2nd[cat]==k] = cat_trans2[cat][k]   

  0%|                                                                                                                      | 0/62 [00:00<?, ?it/s]/tmp/ipykernel_8493/2869851487.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_2nd[cat][train_2nd[cat]==k] = cat_trans[cat][k]
/tmp/ipykernel_8493/2869851487.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_2nd[cat][train_2nd[cat]==k] = cat_trans[cat][k]
/tmp/ipykernel_8493/2869851487.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

 77%|███████████████████████████████████████████████████████████████████████████████████▌                        | 48/62 [00:00<00:00, 118.39it/s]/tmp/ipykernel_8493/2869851487.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_2nd[cat][train_2nd[cat]==k] = cat_trans[cat][k]
/tmp/ipykernel_8493/2869851487.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_2nd[cat][train_2nd[cat]==k] = cat_trans[cat][k]
/tmp/ipykernel_8493/2869851487.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

 14%|███████████████▏                                                                                            | 11/78 [00:00<00:00, 101.60it/s]/tmp/ipykernel_8493/2869851487.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_2nd[cat][train_2nd[cat]==k] = cat_trans[cat][k]
/tmp/ipykernel_8493/2869851487.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_2nd[cat][train_2nd[cat]==k] = cat_trans[cat][k]
/tmp/ipykernel_8493/2869851487.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

 46%|█████████████████████████████████████████████████▊                                                          | 36/78 [00:00<00:00, 119.80it/s]/tmp/ipykernel_8493/2869851487.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_2nd[cat][train_2nd[cat]==k] = cat_trans[cat][k]
/tmp/ipykernel_8493/2869851487.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_2nd[cat][train_2nd[cat]==k] = cat_trans[cat][k]
/tmp/ipykernel_8493/2869851487.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

 82%|████████████████████████████████████████████████████████████████████████████████████████▌                   | 64/78 [00:00<00:00, 128.45it/s]/tmp/ipykernel_8493/2869851487.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_2nd[cat][train_2nd[cat]==k] = cat_trans[cat][k]
/tmp/ipykernel_8493/2869851487.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_2nd[cat][train_2nd[cat]==k] = cat_trans[cat][k]
/tmp/ipykernel_8493/2869851487.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

...

/tmp/ipykernel_8493/2869851487.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_2nd[cat][test_2nd[cat]==k] = cat_trans2[cat][k]
/tmp/ipykernel_8493/2869851487.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_2nd[cat][test_2nd[cat]==k] = cat_trans2[cat][k]
/tmp/ipykernel_8493/2869851487.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_2nd[cat][test_2nd[cat]==k] = cat_trans2[cat][k]
/tmp/ipykernel_8493/28698514

/tmp/ipykernel_8493/2869851487.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_2nd[cat][test_2nd[cat]==k] = cat_trans2[cat][k]
/tmp/ipykernel_8493/2869851487.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_2nd[cat][test_2nd[cat]==k] = cat_trans2[cat][k]
/tmp/ipykernel_8493/2869851487.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_2nd[cat][test_2nd[cat]==k] = cat_trans2[cat][k]
/tmp/ipykernel_8493/28698514

/tmp/ipykernel_8493/2869851487.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_2nd[cat][test_2nd[cat]==k] = cat_trans2[cat][k]
/tmp/ipykernel_8493/2869851487.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_2nd[cat][test_2nd[cat]==k] = cat_trans2[cat][k]
/tmp/ipykernel_8493/2869851487.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_2nd[cat][test_2nd[cat]==k] = cat_trans2[cat][k]
/tmp/ipykernel_8493/28698514

/tmp/ipykernel_8493/2869851487.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_2nd[cat][test_2nd[cat]==k] = cat_trans2[cat][k]
/tmp/ipykernel_8493/2869851487.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_2nd[cat][test_2nd[cat]==k] = cat_trans2[cat][k]
/tmp/ipykernel_8493/2869851487.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_2nd[cat][test_2nd[cat]==k] = cat_trans2[cat][k]
/tmp/ipykernel_8493/28698514

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 295.29it/s]


In [41]:
for col in test_2nd.columns :
    print('col name & nan : ', col,'/',test_2nd[col].isnull().sum())

col name & nan :  application_id / 0
col name & nan :  bank_id / 0
col name & nan :  loan_limit / 0
col name & nan :  loan_rate / 0
col name & nan :  is_applied / 0
col name & nan :  age / 0
col name & nan :  credit_score / 0
col name & nan :  income_type / 0
col name & nan :  desired_amount / 0
col name & nan :  rehabilitation / 0
col name & nan :  n_product / 0
col name & nan :  rate_rank / 0
col name & nan :  diff / 0
col name & nan :  rate_diff_with.min / 0
col name & nan :  relat.rate_diff_with.min / 0
col name & nan :  same_rank_length / 0
col name & nan :  rate_diff_with.above / 0
col name & nan :  relat.rate_diff_with.above / 0
col name & nan :  logdata / 0
col name & nan :  bank_cluster / 0
col name & nan :  cluster_min_rate / 0
col name & nan :  purpose / 0
col name & nan :  employment_type / 0
col name & nan :  houseown_type / 0
col name & nan :  past.is_applied / 0
col name & nan :  m_past.is_applied / 0
col name & nan :  seq / 0
col name & nan :  min.loan_rate / 0
col name

In [49]:
print(train_2nd.columns)
print(valid_2nd.columns)
print(test_2nd.columns)

Index(['application_id', 'bank_id', 'loan_limit', 'loan_rate', 'is_applied',
       'age', 'credit_score', 'income_type', 'desired_amount',
       'rehabilitation', 'n_product', 'rate_rank', 'diff',
       'rate_diff_with.min', 'relat.rate_diff_with.min', 'same_rank_length',
       'rate_diff_with.above', 'relat.rate_diff_with.above', 'logdata',
       'bank_cluster', 'cluster_min_rate', 'past.is_applied',
       'm_past.is_applied', 'seq', 'min.loan_rate', 'product_n',
       'n.limit_over_desire', 'loan_limit_min.rate', 'diff2',
       'limit_over_desire_product', 'yd', 'purpose', 'employment_type',
       'houseown_type'],
      dtype='object')
Index(['application_id', 'bank_id', 'loan_limit', 'loan_rate', 'is_applied',
       'age', 'credit_score', 'income_type', 'desired_amount',
       'rehabilitation', 'n_product', 'rate_rank', 'diff',
       'rate_diff_with.min', 'relat.rate_diff_with.min', 'same_rank_length',
       'rate_diff_with.above', 'relat.rate_diff_with.above', 'logdat

In [42]:
types = valid_2nd.dtypes
target = 'is_applied'
#unused_col = ['set','user_id','application_id','loanapply_insert_time','insert_time','gender','yearly_income',
#             'company_year','existing_loan_cnt','product_id','pred']
unused_col = ['set','application_id','pred','pred2']
cat_col = ['bank_id','purpose','income_type','employment_type','houseown_type','rehabilitation','rate_rank','bank_cluster','cluster_min_rate']
categorical_dims =  {}
for col in valid_2nd.columns:
    if col in cat_col :
        print(col, valid_2nd[col].nunique())
        l_enc = LabelEncoder()
        valid_2nd[col] = l_enc.fit_transform(valid_2nd[col].values)
        categorical_dims[col] = len(l_enc.classes_)
    elif col not in unused_col+[target]:
        valid_2nd.fillna(valid_2nd.loc[:, col].mean(), inplace=True)


# Categorical Embedding을 위해 Categorical 변수의 차원과 idxs를 담음.
features = [ col for col in valid_2nd.columns if col not in unused_col+[target]] 
cat_idxs = [ i for i, f in enumerate(features) if f in cat_col]
cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in cat_col]
#train_shuf = train_shuf.astype({'income_type':'int64','employment_type':'int64','houseown_type':'int64','purpose':'int64'})

bank_id 62
income_type 6
rehabilitation 4
rate_rank 78
bank_cluster 5
cluster_min_rate 6
purpose 8
employment_type 4
houseown_type 4


In [12]:
#def diff(x):
#    if x < -57500000 :
#        return -57500000
#    else : 
#        return x
#train_2nd['diff'] = train_2nd['diff'].apply(diff)
#valid_2nd['diff'] = valid_2nd['diff'].apply(diff)
#test_2nd['diff'] = test_2nd['diff'].apply(diff)
#train_2nd.loc[: ,(train_2nd['diff']< -57500000)] = -57500000
#valid_2nd.loc[: ,(valid_2nd['diff']< -57500000)] = -57500000
#test_2nd.loc[: ,(test_2nd['diff']< -57500000)] = -57500000

In [91]:
for col in train_2nd.columns :
    print('col name & nan : ', col,'/',train_2nd[col].isnull().sum())

col name & nan :  application_id / 0
col name & nan :  loanapply_insert_time / 0
col name & nan :  bank_id / 0
col name & nan :  product_id / 0
col name & nan :  loan_limit / 0
col name & nan :  loan_rate / 0
col name & nan :  is_applied / 0
col name & nan :  user_id / 0
col name & nan :  age / 0
col name & nan :  gender / 0
col name & nan :  insert_time / 0
col name & nan :  credit_score / 0
col name & nan :  yearly_income / 0
col name & nan :  company_year / 0
col name & nan :  desired_amount / 0
col name & nan :  existing_loan_cnt / 0
col name & nan :  n_product / 0
col name & nan :  rate_rank / 0
col name & nan :  diff / 3643175
col name & nan :  rate_diff_with.min / 0
col name & nan :  relat.rate_diff_with.min / 0
col name & nan :  rate_diff_with.above / 0
col name & nan :  relat.rate_diff_with.above / 0
col name & nan :  logdata / 0
col name & nan :  purpose / 0
col name & nan :  income_type / 0
col name & nan :  rehabilitation / 0
col name & nan :  houseown_type / 0
col name & n

In [90]:
for col in test_2nd.columns :
    print('col name & nan : ', col,'/',test_2nd[col].isnull().sum())

col name & nan :  application_id / 0
col name & nan :  loanapply_insert_time / 0
col name & nan :  bank_id / 0
col name & nan :  product_id / 0
col name & nan :  loan_limit / 0
col name & nan :  loan_rate / 0
col name & nan :  is_applied / 0
col name & nan :  user_id / 0
col name & nan :  age / 0
col name & nan :  gender / 0
col name & nan :  insert_time / 0
col name & nan :  credit_score / 0
col name & nan :  yearly_income / 0
col name & nan :  company_year / 0
col name & nan :  desired_amount / 0
col name & nan :  existing_loan_cnt / 0
col name & nan :  n_product / 0
col name & nan :  rate_rank / 0
col name & nan :  diff / 0
col name & nan :  rate_diff_with.min / 0
col name & nan :  relat.rate_diff_with.min / 0
col name & nan :  rate_diff_with.above / 0
col name & nan :  relat.rate_diff_with.above / 0
col name & nan :  logdata / 0
col name & nan :  purpose / 0
col name & nan :  income_type / 0
col name & nan :  rehabilitation / 0
col name & nan :  houseown_type / 0
col name & nan :  

In [96]:
test_2nd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 818591 entries, 0 to 818590
Data columns (total 35 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   application_id              818591 non-null  int64  
 1   bank_id                     818591 non-null  int64  
 2   loan_limit                  818591 non-null  float64
 3   loan_rate                   818591 non-null  float64
 4   is_applied                  818591 non-null  int64  
 5   age                         818591 non-null  int64  
 6   credit_score                818591 non-null  int64  
 7   income_type                 818591 non-null  object 
 8   desired_amount              818591 non-null  float64
 9   rehabilitation              818591 non-null  int64  
 10  n_product                   818591 non-null  int64  
 11  rate_rank                   818591 non-null  int64  
 12  diff                        818591 non-null  float64
 13  rate_diff_with

In [43]:
train_2nd = train_2nd.astype({'income_type':'float32','purpose':'float32','employment_type':'float32','houseown_type':'float32'})
valid_2nd = valid_2nd.astype({'income_type':'float32','purpose':'float32','employment_type':'float32','houseown_type':'float32'})
test_2nd = test_2nd.astype({'income_type':'float32','purpose':'float32','employment_type':'float32','houseown_type':'float32'})

In [64]:
train_2nd.to_csv('/Data2/folder_ti/data/2nd/train_2nd1.csv')
valid_2nd.to_csv('/Data2/folder_ti/data/2nd/valid_2nd1.csv')
test_2nd.to_csv('/Data2/folder_ti/data/2nd/test_2nd1.csv')

In [44]:

X_train = train_2nd.loc[:,features].values
y_train = train_2nd.loc[:,target].values

X_valid = test_2nd.loc[:,features].values
y_valid = test_2nd.loc[:,target].values


In [45]:
X_train.shape

(3807822, 32)

In [46]:
X_valid.shape

(818591, 32)

In [52]:
features

['bank_id',
 'loan_limit',
 'loan_rate',
 'age',
 'credit_score',
 'income_type',
 'desired_amount',
 'rehabilitation',
 'n_product',
 'rate_rank',
 'diff',
 'rate_diff_with.min',
 'relat.rate_diff_with.min',
 'same_rank_length',
 'rate_diff_with.above',
 'relat.rate_diff_with.above',
 'logdata',
 'bank_cluster',
 'cluster_min_rate',
 'past.is_applied',
 'm_past.is_applied',
 'seq',
 'min.loan_rate',
 'product_n',
 'n.limit_over_desire',
 'loan_limit_min.rate',
 'diff2',
 'limit_over_desire_product',
 'yd',
 'purpose',
 'employment_type',
 'houseown_type']

In [53]:
cat_idxs

[0, 5, 7, 9, 17, 18, 29, 30, 31]

In [54]:
cat_dims

[62, 6, 4, 78, 5, 6, 8, 4, 4]

In [47]:
import torch
DEVICE = 'cuda:2' if torch.cuda.is_available() else 'cpu'
print("Using {}".format(DEVICE))


params = {'n_d' : 4,
         'n_steps' :5,        # tree에 depth
         'gamma':1.5,
         'n_independent':2,
         'cat_idxs':cat_idxs,
         'cat_dims':cat_dims,
         'n_shared':2,
         'cat_emb_dim':1,
         'lambda_sparse':1e-4,
         'momentum':0.5,
         'clip_value':2.,
         'optimizer_fn':torch.optim.Adam,
         'optimizer_params':dict(lr=2e-2),
         'scheduler_params':{"gamma": 0.95,
                         "step_size": 20},
         'scheduler_fn':torch.optim.lr_scheduler.StepLR,
         'epsilon':1e-15,
         'device_name':DEVICE,
         'mask_type':'sparsemax'}

clf = TabNetClassifier(**params)
s = time.time()

clf.fit(X_train=X_train, y_train=y_train,eval_set=[(X_valid, y_valid)],
    max_epochs=30 , patience=20,
    batch_size=10240, virtual_batch_size=1280,
    num_workers=0,
    weights=1,
    drop_last=False
)

print('TabNet gpu sec : ',time.time()- s)


pred_valid = clf.predict(X_valid)


f1_valid = f1_score(y_valid, pred_valid) # y_true : 정답 값, y_pred : 예측 값 
print("Valid F1 score: {}".format(f1_valid))
 

from sklearn.metrics import roc_auc_score 
roc_auc = roc_auc_score(y_valid, pred_valid)
print("Valid ROC AUC Score: {}".format(roc_auc))

Using cuda:2


/opt/conda/lib/python3.8/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cuda:2
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.45646 | val_0_auc: 0.88388 |  0:01:11s
epoch 1  | loss: 0.40975 | val_0_auc: 0.89343 |  0:02:31s
epoch 2  | loss: 0.39711 | val_0_auc: 0.90273 |  0:03:42s
epoch 3  | loss: 0.38396 | val_0_auc: 0.90682 |  0:04:52s
epoch 4  | loss: 0.371   | val_0_auc: 0.91247 |  0:06:01s
epoch 5  | loss: 0.36351 | val_0_auc: 0.9144  |  0:07:14s
epoch 6  | loss: 0.35751 | val_0_auc: 0.91787 |  0:08:24s
epoch 7  | loss: 0.35073 | val_0_auc: 0.91912 |  0:09:34s
epoch 8  | loss: 0.34693 | val_0_auc: 0.91749 |  0:10:47s
epoch 9  | loss: 0.34564 | val_0_auc: 0.91988 |  0:11:59s
epoch 10 | loss: 0.34375 | val_0_auc: 0.92197 |  0:13:14s
epoch 11 | loss: 0.34268 | val_0_auc: 0.92041 |  0:14:26s
epoch 12 | loss: 0.34198 | val_0_auc: 0.92266 |  0:15:36s
epoch 13 | loss: 0.34084 | val_0_auc: 0.92324 |  0:16:50s
epoch 14 | loss: 0.34088 | val_0_auc: 0.92336 |  0:18:01s
epoch 15 | loss: 0.34092 | val_0_auc: 0.9235  |  0:19:13s
epoch 16 | loss: 0.33963 | val_0_auc: 0.92365 |  0:20:23s
epoch 17 | los

/opt/conda/lib/python3.8/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


TabNet gpu sec :  2462.5988121032715
Valid F1 score: 0.5596753981404622
Valid ROC AUC Score: 0.843345615317697


## 0.34, 0.85

In [48]:
pred_prob_valid = clf.predict_proba(X_valid)

def thres_optimization(threshold, array) :
    pred = np.ceil(array[:,1]-threshold)
    return pred

score_valid = [0]
best_thres_valid = 0
for i in range(1,100) :
    pre_valid = thres_optimization(i/100,pred_prob_valid)
    
    f1_valid = f1_score(y_valid, pre_valid)
    
    score_valid.append(f1_valid)
    
best_thres_valid = score_valid.index(max(score_valid))/100
print(max(score_valid))
print(best_thres_valid)

    

0.6312256883532912
0.76


In [20]:
second.shape[0]

4763080

In [22]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(test_2nd.is_applied.values, test_2nd.pred2.values))

[[670341  46513]
 [ 35836  65901]]


In [49]:
test_id_set = set(test_2nd.application_id.tolist())
valid_id_set = set(valid_2nd.application_id.tolist())
test_else_id_set = list(valid_id_set - test_id_set)
test_else_set = valid_2nd[valid_2nd['application_id'].isin(test_else_id_set)]


In [50]:
valid_2nd.is_applied.value_counts()

0    843984
1    111274
Name: is_applied, dtype: int64

In [51]:
test_else_set.is_applied.value_counts()

0    127130
1      9537
Name: is_applied, dtype: int64

In [52]:
test_2nd.is_applied.value_counts()

0    716854
1    101737
Name: is_applied, dtype: int64

In [59]:

test_2nd['pred2'] = thres_optimization(0.76,pred_prob_valid)
test_else_set['pred2'] = thres_optimization(0.76,clf.predict_proba(test_else_set.loc[:,features].values))

/tmp/ipykernel_8493/3313710319.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_else_set['pred2'] = thres_optimization(0.76,clf.predict_proba(test_else_set.loc[:,features].values))


In [56]:
test_else_set['pred2'] = 0

/tmp/ipykernel_8493/3565325071.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_else_set['pred2'] = 0


In [60]:
final_2nd2 = pd.concat([test_2nd,test_else_set], ignore_index=True)

In [89]:
final_2nd2.head()

,application_id,bank_id,loan_limit,loan_rate,is_applied,age,credit_score,income_type,desired_amount,rehabilitation,...,n.limit_over_desire,loan_limit_min.rate,diff2,limit_over_desire_product,yd,purpose,employment_type,houseown_type,pred2,pred
0,37,39,10000000.0,7.3,1,20,730,3,5000000.0,3,...,1,10000000.0,1.0,1.0,0.250000,7,0,2,1.0,NaN
1,76,50,14000000.0,4.2,1,54,670,0,20000000.0,2,...,15,14000000.0,-0.3,0.5,0.571429,7,0,2,1.0,NaN
2,76,47,9000000.0,6.5,0,54,670,0,20000000.0,2,...,15,14000000.0,-0.3,0.5,0.571429,7,0,2,-0.0,NaN
3,76,57,29000000.0,6.6,0,54,670,0,20000000.0,2,...,15,14000000.0,-0.3,0.5,0.571429,7,0,2,-0.0,NaN
4,76,46,9000000.0,6.7,0,54,670,0,20000000.0,2,...,15,14000000.0,-0.3,0.5,0.571429,7,0,2,-0.0,NaN


In [18]:
final_2nd = pd.merge(valid_2nd, test_2nd.loc[:,['application_id','pred2']], on='application_id', how='left')

In [14]:
final_2nd.fillna(0, inplace=True)

In [16]:
final_2nd.head()

,application_id,bank_id,loan_limit,loan_rate,is_applied,age,credit_score,desired_amount,n_product,rate_rank,...,rate_diff_with.above,relat.rate_diff_with.above,logdata,purpose,income_type,rehabilitation,houseown_type,employment_type,pred,pred2
0,37,39,10000000.0,7.3,1,20,730,5000000.0,1,1.0,...,0.0,0.0,7,3,3,0,2,1,1.0,1.0
1,76,50,14000000.0,4.2,1,54,670,20000000.0,32,1.0,...,0.0,0.0,9,3,4,3,2,1,1.0,1.0
2,76,50,14000000.0,4.2,1,54,670,20000000.0,32,1.0,...,0.0,0.0,9,3,4,3,2,1,1.0,-0.0
3,76,50,14000000.0,4.2,1,54,670,20000000.0,32,1.0,...,0.0,0.0,9,3,4,3,2,1,1.0,-0.0
4,76,50,14000000.0,4.2,1,54,670,20000000.0,32,1.0,...,0.0,0.0,9,3,4,3,2,1,1.0,-0.0


## 2차만

In [61]:
print('2nd_classification_f1_score : ',f1_score(final_2nd2.is_applied.values, final_2nd2.pred2.values))

2nd_classification_f1_score :  0.6302857690331234


## 1,2 차 모두

In [58]:
print('2nd_classification_f1_score : ',f1_score(final_2nd2.is_applied.values, final_2nd2.pred2.values))

2nd_classification_f1_score :  0.6037763379279842


In [17]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(final_2nd.is_applied.values, final_2nd.pred2.values))

[[20243279  1077681]
 [ 1150647   174887]]


## Smote 2nd_classification

In [66]:
from imblearn.over_sampling import SMOTE

smote_sample = SMOTE(random_state=0)
X_train_over,y_train_over = smote_sample.fit_resample(X_train,y_train)

In [67]:
import torch
DEVICE = 'cuda:3' if torch.cuda.is_available() else 'cpu'
print("Using {}".format(DEVICE))

params = {'n_d' : 4,
         'n_steps' :5,        # tree에 depth
         'gamma':1.5,
         'n_independent':2,
          'cat_idxs':cat_idxs,
          'cat_dims':cat_dims,
         'n_shared':2,
         'cat_emb_dim':1,
         'lambda_sparse':1e-4,
         'momentum':0.5,
         'clip_value':2.,
         'optimizer_fn':torch.optim.Adam,
         'optimizer_params':dict(lr=2e-2),
         'scheduler_params':{"gamma": 0.95,
                         "step_size": 20},
         'scheduler_fn':torch.optim.lr_scheduler.StepLR,
         'epsilon':1e-15,
         'device_name':DEVICE,
         'mask_type':'sparsemax'}

clf = TabNetClassifier(**params
)
s = time.time()

clf.fit(X_train=X_train_over, y_train=y_train_over,eval_set=[(X_valid, y_valid)],
    max_epochs=30 , patience=20,
    batch_size=10240, virtual_batch_size=1280,
    num_workers=0,
    weights=1,
    drop_last=False
)

print('TabNet gpu sec : ',time.time()- s)


pred_valid = clf.predict(X_valid)


f1_valid = f1_score(y_valid, pred_valid) # y_true : 정답 값, y_pred : 예측 값 
print("Valid F1 score: {}".format(f1_valid))
 

from sklearn.metrics import roc_auc_score 
roc_auc = roc_auc_score(y_valid, pred_valid)
print("Valid ROC AUC Score: {}".format(roc_auc))

Using cuda:3


/opt/conda/lib/python3.8/site-packages/pytorch_tabnet/abstract_model.py:75: UserWarning: Device used : cuda:3
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.38092 | val_0_auc: 0.8663  |  0:02:18s
epoch 1  | loss: 0.31561 | val_0_auc: 0.86826 |  0:04:37s
epoch 2  | loss: 0.31034 | val_0_auc: 0.86369 |  0:07:09s
epoch 3  | loss: 0.31277 | val_0_auc: 0.84191 |  0:09:55s
epoch 4  | loss: 0.30593 | val_0_auc: 0.87058 |  0:13:11s
epoch 5  | loss: 0.293   | val_0_auc: 0.75227 |  0:16:32s
epoch 6  | loss: 0.28164 | val_0_auc: 0.85365 |  0:19:45s
epoch 7  | loss: 0.27791 | val_0_auc: 0.86822 |  0:23:06s
epoch 8  | loss: 0.28427 | val_0_auc: 0.87087 |  0:26:27s
epoch 9  | loss: 0.28251 | val_0_auc: 0.84838 |  0:29:37s
epoch 10 | loss: 0.26741 | val_0_auc: 0.76546 |  0:32:38s
epoch 11 | loss: 0.26467 | val_0_auc: 0.71643 |  0:35:42s
epoch 12 | loss: 0.26379 | val_0_auc: 0.73941 |  0:38:51s
epoch 13 | loss: 0.27603 | val_0_auc: 0.85649 |  0:41:46s
epoch 14 | loss: 0.27164 | val_0_auc: 0.86915 |  0:44:50s
epoch 15 | loss: 0.28202 | val_0_auc: 0.83843 |  0:47:56s
epoch 16 | loss: 0.27437 | val_0_auc: 0.87065 |  0:51:06s
epoch 17 | los

/opt/conda/lib/python3.8/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


KeyboardInterrupt: 

In [68]:
pred_prob_valid = clf.predict_proba(X_valid)

def thres_optimization(threshold, array) :
    pred = np.ceil(array[:,1]-threshold)
    return pred

score_valid = [0]
best_thres_valid = 0
for i in range(1,100) :
    pre_valid = thres_optimization(i/100,pred_prob_valid)
    
    f1_valid = f1_score(y_valid, pre_valid)
    
    score_valid.append(f1_valid)
    
best_thres_valid = score_valid.index(max(score_valid))/100
print(max(score_valid))
print(best_thres_valid)

0.555187436926182
0.66


In [69]:

test_id_set = set(test_2nd.application_id.tolist())
valid_id_set = set(valid_2nd.application_id.tolist())
test_else_id_set = list(valid_id_set - test_id_set)
test_else_set = valid_2nd[valid_2nd['application_id'].isin(test_else_id_set)]


In [70]:

test_2nd['pred2'] = thres_optimization(0.76,pred_prob_valid)
test_else_set['pred2'] = thres_optimization(0.76,clf.predict_proba(test_else_set.loc[:,features].values))

/tmp/ipykernel_8493/3313710319.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_else_set['pred2'] = thres_optimization(0.76,clf.predict_proba(test_else_set.loc[:,features].values))


In [73]:
test_else_set['pred2'] = 0

/tmp/ipykernel_8493/3565325071.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_else_set['pred2'] = 0


In [74]:
final_2nd2 = pd.concat([test_2nd,test_else_set], ignore_index=True)

## 2차만

In [72]:
print('2nd_classification_f1_score : ',f1_score(final_2nd2.is_applied.values, final_2nd2.pred2.values))

2nd_classification_f1_score :  0.5332487258516146


## 1,2 차 모두

In [75]:
print('2nd_classification_f1_score : ',f1_score(final_2nd2.is_applied.values, final_2nd2.pred2.values))

2nd_classification_f1_score :  0.513813222458435


In [ ]:
def weighted_mean(x,y,thres) :
    for i in range(100) :
        

In [2]:
a = np.array([0,1,1])
b = np.array([1,1,0])
print((a*80+b*20)/100)

[0.2 1.  0.8]


In [7]:
import pandas as pd
sub = pd.read_csv('/Data2/folder_ti/data/submission/submission/데이터분석분야_퓨처스부문_빅월드콘팀_평가데이터.csv',index_col = 0)

In [8]:
sub.head()

,application_id,product_id,is_applied
0,4,220,1.0
1,4,191,1.0
2,8,29,1.0
3,8,159,-0.0
4,8,85,1.0


In [9]:
sub.to_csv('/Data2/folder_ti/data/submission/submission/데이터분석분야_퓨처스부문_빅월드콘팀_평가데이터_.csv',index=False)

In [10]:
sub.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3255194 entries, 0 to 3255193
Data columns (total 3 columns):
 #   Column          Dtype  
---  ------          -----  
 0   application_id  int64  
 1   product_id      int64  
 2   is_applied      float64
dtypes: float64(1), int64(2)
memory usage: 99.3 MB


In [13]:
su = pd.read_csv('/Data2/folder_ti/data/submission/submission.csv')

In [14]:
su.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3255194 entries, 0 to 3255193
Data columns (total 3 columns):
 #   Column          Dtype  
---  ------          -----  
 0   application_id  int64  
 1   product_id      int64  
 2   is_applied      float64
dtypes: float64(1), int64(2)
memory usage: 74.5 MB
